<h2>The Battle Of Neighborhoods | Notebook

<h3>This whole process will be including Foursquare API as the source of information

<h4>Questions to be answered:</h4>
    <li> What are some best hotels in New York City?</li>
    <li> Visualize parts of New York City that has best hotels</li> 
    <li> Which hotels are cheap and have a good rating?</li> 
    <li> Best Hotels which are nearby airport</li>
    <li> Hotels which are nearby best restaurants</li> 
    <li> Best Hotels which are nearby Subway Station</li> 

<h4>Importing Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

<h3>Getting the data of Latitude and Longitude of New York City

In [8]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Searching for hotels

In [10]:
search_query = 'Hotel'
radius = 500
LIMIT = 100
CLIENT_ID = 'Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'FO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=SOX21PX0BG3KXO41DN2OHISIO3HFPN1Z5IKMB1KQVC4H023Y&client_secret=MRRCVJ4XTAOLO2MCLJRZCO0PIHR3OCZYQV41PRME3BPEPAB2&ll=40.7127281,-74.0060152&v=20180605&query=Hotel&radius=500&limit=100'

In [11]:
results = requests.get(url).json()

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,57c640ad498e74977f98372f,Four Seasons Hotel New York Downtown,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1594553146,False,27 Barclay St,Broadway,40.712612,-74.009380,"[{'label': 'display', 'lat': 40.71261246501983...",284,10007,US,New York,NY,United States,"[27 Barclay St (Broadway), New York, NY 10007,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56d8c0f8498edb854f926e6a,"The Beekman, A Thompson Hotel","[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1594553146,False,123 Nassau St,Beekman St,40.711173,-74.006702,"[{'label': 'display', 'lat': 40.7111725, 'lng'...",182,10038,US,New York,NY,United States,"[123 Nassau St (Beekman St), New York, NY 1003...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4ae77379f964a5206cab21e3,Duane Street Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1594553146,False,130 Duane St,Church St,40.715858,-74.007446,"[{'label': 'display', 'lat': 40.7158584594727,...",368,10013,US,New York,NY,United States,"[130 Duane St (Church St), New York, NY 10013,...",40822074,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4a6e3e75f964a5204ed41fe3,"Gild Hall, A Thompson Hotel","[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1594553146,False,15 Gold St,at Platt St,40.707887,-74.007103,"[{'label': 'display', 'lat': 40.7078871, 'lng'...",546,10038,US,New York,NY,United States,"[15 Gold St (at Platt St), New York, NY 10038,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5a91fa58e679bc57ca94db72,Artezen Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1594553146,False,24 John St,NaN,40.709585,-74.008780,"[{'label': 'display', 'lat': 40.709585, 'lng':...",420,10038,US,New York,NY,United States,"[24 John St, New York, NY 10038, United States]",563082986,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
filtered_columns = ['name', 'categories'] + [col for col in nearby_venues.columns if col.startswith('location')]+ ['id']
filtered_df = nearby_venues.loc[:,filtered_columns]

filtered_df['categories'] = filtered_df.apply(get_category_type, axis=1)

filtered_df.columns = [col.split('.')[-1].capitalize() for col in filtered_df.columns]

filtered_df.head()


,Name,Categories,Address,Crossstreet,Lat,Lng,Labeledlatlngs,Distance,Postalcode,Cc,City,State,Country,Formattedaddress,Neighborhood,Id
0,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,Broadway,40.712612,-74.009380,"[{'label': 'display', 'lat': 40.71261246501983...",284,10007,US,New York,NY,United States,"[27 Barclay St (Broadway), New York, NY 10007,...",NaN,57c640ad498e74977f98372f
1,"The Beekman, A Thompson Hotel",Hotel,123 Nassau St,Beekman St,40.711173,-74.006702,"[{'label': 'display', 'lat': 40.7111725, 'lng'...",182,10038,US,New York,NY,United States,"[123 Nassau St (Beekman St), New York, NY 1003...",NaN,56d8c0f8498edb854f926e6a
2,Duane Street Hotel,Hotel,130 Duane St,Church St,40.715858,-74.007446,"[{'label': 'display', 'lat': 40.7158584594727,...",368,10013,US,New York,NY,United States,"[130 Duane St (Church St), New York, NY 10013,...",NaN,4ae77379f964a5206cab21e3
3,"Gild Hall, A Thompson Hotel",Hotel,15 Gold St,at Platt St,40.707887,-74.007103,"[{'label': 'display', 'lat': 40.7078871, 'lng'...",546,10038,US,New York,NY,United States,"[15 Gold St (at Platt St), New York, NY 10038,...",NaN,4a6e3e75f964a5204ed41fe3
4,Artezen Hotel,Hotel,24 John St,NaN,40.709585,-74.008780,"[{'label': 'display', 'lat': 40.709585, 'lng':...",420,10038,US,New York,NY,United States,"[24 John St, New York, NY 10038, United States]",NaN,5a91fa58e679bc57ca94db72


In [13]:
filtered_df.drop(['Cc', 'City', 'Country', 'Crossstreet', 'Distance', 'Formattedaddress',\
                                        'Labeledlatlngs','Neighborhood'], axis=1, inplace=True)

filtered_df.dropna(axis=0, inplace=True)

filtered_df.head()

,Name,Categories,Address,Lat,Lng,Postalcode,State,Id
0,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,40.712612,-74.009380,10007,NY,57c640ad498e74977f98372f
1,"The Beekman, A Thompson Hotel",Hotel,123 Nassau St,40.711173,-74.006702,10038,NY,56d8c0f8498edb854f926e6a
2,Duane Street Hotel,Hotel,130 Duane St,40.715858,-74.007446,10013,NY,4ae77379f964a5206cab21e3
3,"Gild Hall, A Thompson Hotel",Hotel,15 Gold St,40.707887,-74.007103,10038,NY,4a6e3e75f964a5204ed41fe3
4,Artezen Hotel,Hotel,24 John St,40.709585,-74.008780,10038,NY,5a91fa58e679bc57ca94db72


In [14]:
filtered_df.drop(['State', 'Postalcode'], axis=1, inplace=True)

In [15]:
filtered_df.head()

,Name,Categories,Address,Lat,Lng,Id
0,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,40.712612,-74.009380,57c640ad498e74977f98372f
1,"The Beekman, A Thompson Hotel",Hotel,123 Nassau St,40.711173,-74.006702,56d8c0f8498edb854f926e6a
2,Duane Street Hotel,Hotel,130 Duane St,40.715858,-74.007446,4ae77379f964a5206cab21e3
3,"Gild Hall, A Thompson Hotel",Hotel,15 Gold St,40.707887,-74.007103,4a6e3e75f964a5204ed41fe3
4,Artezen Hotel,Hotel,24 John St,40.709585,-74.008780,5a91fa58e679bc57ca94db72


In [16]:
filtered_df['Categories'].unique()

array(['Hotel', 'Gym / Fitness Center', 'American Restaurant',
       'French Restaurant', 'Hotel Bar'], dtype=object)

<h3>We want to include categories containing Hotel

In [17]:
filtered_df2 = filtered_df.loc[filtered_df['Categories'].str.contains('Hotel')]
filtered_df2

,Name,Categories,Address,Lat,Lng,Id
0,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,40.712612,-74.009380,57c640ad498e74977f98372f
1,"The Beekman, A Thompson Hotel",Hotel,123 Nassau St,40.711173,-74.006702,56d8c0f8498edb854f926e6a
2,Duane Street Hotel,Hotel,130 Duane St,40.715858,-74.007446,4ae77379f964a5206cab21e3
3,"Gild Hall, A Thompson Hotel",Hotel,15 Gold St,40.707887,-74.007103,4a6e3e75f964a5204ed41fe3
4,Artezen Hotel,Hotel,24 John St,40.709585,-74.008780,5a91fa58e679bc57ca94db72
5,Smyth Hotel,Hotel,85 W Broadway,40.715144,-74.009183,49efcc88f964a52006691fe3
6,Cosmopolitan Hotel - TriBeCa,Hotel,95 W Broadway,40.715622,-74.009115,4a3a9952f964a52001a01fe3
7,The Frederick Hotel,Hotel,95 W Broadway,40.715675,-74.009047,59e391c3e1f22816d1b09cf5
18,The Frederick Hotel,Hotel,95 West Broadway,40.715460,-74.009383,4bed7bbebac3c9b60157fee9
20,Smythe Thompson Hotel,Hotel,86 W Broadway,40.715323,-74.009654,52855bf511d21362ef6f329f


In [18]:
hotel_df = filtered_df2.drop_duplicates(subset='Name')
hotel_df.head()

,Name,Categories,Address,Lat,Lng,Id
0,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,40.712612,-74.009380,57c640ad498e74977f98372f
1,"The Beekman, A Thompson Hotel",Hotel,123 Nassau St,40.711173,-74.006702,56d8c0f8498edb854f926e6a
2,Duane Street Hotel,Hotel,130 Duane St,40.715858,-74.007446,4ae77379f964a5206cab21e3
3,"Gild Hall, A Thompson Hotel",Hotel,15 Gold St,40.707887,-74.007103,4a6e3e75f964a5204ed41fe3
4,Artezen Hotel,Hotel,24 John St,40.709585,-74.008780,5a91fa58e679bc57ca94db72


<h4>Ratings and pricings were recorded manually from google so that the data is more reliable 

In [27]:
ratings = [9.4, 9.2, 8.4, 9.0, 8.9, 8.5, 9.3, 8.6, 8.7, 8.0, 8.4, 8.8, 8.0]
#Pricing is kept in dollars
price = [454, 100, 160, 190, 90, 200, 120, 151, 100, 182, 250, 100, 130]

In [28]:
hotel_df['Price'] = price
hotel_df['Ratings'] = ratings
hotel_df.reset_index(inplace=True)
hotel_df.drop('index', axis=1, inplace=True)
hotel_df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

,Name,Categories,Address,Lat,Lng,Id,Price,Ratings
0,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,40.712612,-74.009380,57c640ad498e74977f98372f,454,9.4
1,"The Beekman, A Thompson Hotel",Hotel,123 Nassau St,40.711173,-74.006702,56d8c0f8498edb854f926e6a,100,9.2
2,Duane Street Hotel,Hotel,130 Duane St,40.715858,-74.007446,4ae77379f964a5206cab21e3,160,8.4
3,"Gild Hall, A Thompson Hotel",Hotel,15 Gold St,40.707887,-74.007103,4a6e3e75f964a5204ed41fe3,190,9.0
4,Artezen Hotel,Hotel,24 John St,40.709585,-74.008780,5a91fa58e679bc57ca94db72,90,8.9


<h2>Extracting the hotels that have rating greater than or equal to 9.0 (Best Hotels)

In [29]:
best_hotels_df = hotel_df[hotel_df['Ratings'] >= 9.0]
best_hotels_df.head()

,Name,Categories,Address,Lat,Lng,Id,Price,Ratings
0,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,40.712612,-74.009380,57c640ad498e74977f98372f,454,9.4
1,"The Beekman, A Thompson Hotel",Hotel,123 Nassau St,40.711173,-74.006702,56d8c0f8498edb854f926e6a,100,9.2
3,"Gild Hall, A Thompson Hotel",Hotel,15 Gold St,40.707887,-74.007103,4a6e3e75f964a5204ed41fe3,190,9.0
6,Cosmopolitan Hotel - TriBeCa,Hotel,95 W Broadway,40.715622,-74.009115,4a3a9952f964a52001a01fe3,120,9.3


<h2>Visualizing the best hotels on a map

In [34]:
new_york_map = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, name, rating, price in zip(best_hotels_df['Lat'], 
                                         best_hotels_df['Lng'], 
                                         best_hotels_df['Name'], 
                                         best_hotels_df['Ratings'],
                                         best_hotels_df['Price']
                                        ):
    label = '{}, Average Cost is ${}'.format(name, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(new_york_map)
new_york_map

<h3>Extracting the hotels that are cheap (Below 150 dollars) and have a good rating (above 8.0)

In [31]:
cheap_good_hotels = hotel_df.loc[(hotel_df['Ratings'] > 8.0) & (hotel_df['Price'] < 150)]
cheap_good_hotels.head()

,Name,Categories,Address,Lat,Lng,Id,Price,Ratings
1,"The Beekman, A Thompson Hotel",Hotel,123 Nassau St,40.711173,-74.006702,56d8c0f8498edb854f926e6a,100,9.2
4,Artezen Hotel,Hotel,24 John St,40.709585,-74.008780,5a91fa58e679bc57ca94db72,90,8.9
6,Cosmopolitan Hotel - TriBeCa,Hotel,95 W Broadway,40.715622,-74.009115,4a3a9952f964a52001a01fe3,120,9.3
8,Smythe Thompson Hotel,Hotel,86 W Broadway,40.715323,-74.009654,52855bf511d21362ef6f329f,100,8.7
11,Holiday Inn New York City - Wall Street,Hotel,51 Nassau St,40.708977,-74.009123,4b0017f7f964a520bb3a22e3,100,8.8


In [33]:
cheap_good_hotels_map = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, name, rating, price in zip(cheap_good_hotels['Lat'], 
                                         cheap_good_hotels['Lng'], 
                                         cheap_good_hotels['Name'], 
                                         cheap_good_hotels['Ratings'],
                                         cheap_good_hotels['Price']
                                        ):
    label = '{}, Average Cost is ${}'.format(name, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(cheap_good_hotels_map)
cheap_good_hotels_map

In [35]:
search_query = 'Airport'

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=SOX21PX0BG3KXO41DN2OHISIO3HFPN1Z5IKMB1KQVC4H023Y&client_secret=MRRCVJ4XTAOLO2MCLJRZCO0PIHR3OCZYQV41PRME3BPEPAB2&ll=40.7127281,-74.0060152&v=20180605&query=Airport&radius=500&limit=100'

In [36]:
results = requests.get(url).json()['response']['venues']

airport = json_normalize(results) # flatten JSON

airport.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,56c7e885498e632787139b41,NYC Airport Limo,"[{'id': '54541b70498ea6ccd0204bff', 'name': 'T...",v-1594554392,False,225 Broadway,40.712362,-74.010346,"[{'label': 'display', 'lat': 40.71236189705591...",367,10007,US,New York,NY,United States,"[225 Broadway, New York, NY 10007, United States]"


In [37]:

filtered_columns = ['name', 'categories'] + [col for col in airport.columns if col.startswith('location')]+ ['id']
filtered_df = airport.loc[:,filtered_columns]

filtered_df['categories'] = filtered_df.apply(get_category_type, axis=1)

filtered_df.columns = [col.split('.')[-1].capitalize() for col in filtered_df.columns]

filtered_df.head()

,Name,Categories,Address,Lat,Lng,Labeledlatlngs,Distance,Postalcode,Cc,City,State,Country,Formattedaddress,Id
0,NYC Airport Limo,Transportation Service,225 Broadway,40.712362,-74.010346,"[{'label': 'display', 'lat': 40.71236189705591...",367,10007,US,New York,NY,United States,"[225 Broadway, New York, NY 10007, United States]",56c7e885498e632787139b41


In [41]:
try:
    filtered_df.drop(['Cc', 'City', 'Country', 'Distance', 'Formattedaddress',\
                                            'Labeledlatlngs', 'State', 'Postalcode', 'Id'], axis=1, inplace=True)
except:
    pass


In [42]:
filtered_df.head()

,Name,Categories,Address,Lat,Lng
0,NYC Airport Limo,Transportation Service,225 Broadway,40.712362,-74.010346


<h3>Visualizing hotels near the Airport

In [44]:
hotels_nearAirport = folium.Map(location=[latitude, longitude], zoom_start=14)

#Adding airport to the map
for lat, lng, name in zip(filtered_df['Lat'], 
                          filtered_df['Lng'], 
                          filtered_df['Name'] 
                          ):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='Red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(hotels_nearAirport)

#Adding hotels to the map
for lat, lng, name, price in zip(hotel_df['Lat'], 
                                 hotel_df['Lng'], 
                                 hotel_df['Name'], 
                                 hotel_df['Price']
                                 ):
    label = '{}, Average Cost is ${}'.format(name, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(hotels_nearAirport)
    
hotels_nearAirport

<h4>As per the results, 2 hotels were found near the airport which are 'Four Seasons Hotel New York Downtown' (454 dollars) and 'Millennium Hilton' (250 dollars)

<h3>Now lets search hotels near the best restaurant

In [45]:
search_query = 'Restaurant'

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=SOX21PX0BG3KXO41DN2OHISIO3HFPN1Z5IKMB1KQVC4H023Y&client_secret=MRRCVJ4XTAOLO2MCLJRZCO0PIHR3OCZYQV41PRME3BPEPAB2&ll=40.7127281,-74.0060152&v=20180605&query=Restaurant&radius=500&limit=100'

In [48]:
results = requests.get(url).json()['response']['venues']

    
restaurants_df = json_normalize(results) # flatten JSON

filtered_columns = ['name', 'categories'] + [col for col in restaurants_df.columns if col.startswith('location')]+ ['id']
filtered_df = restaurants_df.loc[:,filtered_columns]

filtered_df['categories'] = filtered_df.apply(get_category_type, axis=1)

filtered_df.columns = [col.split('.')[-1].capitalize() for col in filtered_df.columns]

filtered_df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,Name,Categories,Address,Crossstreet,Lat,Lng,Labeledlatlngs,Distance,Postalcode,Cc,City,State,Country,Formattedaddress,Id
0,TJ Byrnes Bar and Restaurant,Restaurant,77 Fulton St,Gold St,40.709233,-74.003747,"[{'label': 'display', 'lat': 40.70923312629616...",433,10038,US,New York,NY,United States,"[77 Fulton St (Gold St), New York, NY 10038, U...",4b4fdfc8f964a520801827e3
1,Mudville Restaurant & Tap House,Wings Joint,126 Chambers St,btwn W Broadway & Church St,40.715336,-74.008881,"[{'label': 'display', 'lat': 40.71533575723845...",377,10007,US,New York,NY,United States,[126 Chambers St (btwn W Broadway & Church St)...,45e5c256f964a52046431fe3
2,New Shezan Restaurant,Middle Eastern Restaurant,183 Church St,btwn Duane & Reade St.,40.715789,-74.007227,"[{'label': 'display', 'lat': 40.71578949233164...",355,10007,US,New York,NY,United States,"[183 Church St (btwn Duane & Reade St.), New Y...",4c4890da1879c9b6cce6e143
3,Win Won Restaurant,Chinese Restaurant,10 Liberty Pl,Maiden Lane,40.709193,-74.009344,"[{'label': 'display', 'lat': 40.709193, 'lng':...",483,10038,US,New York,NY,United States,"[10 Liberty Pl (Maiden Lane), New York, NY 100...",4b88199af964a520e6df31e3
4,Amore's Pizza Restaurant,Pizza Place,147 Chambers St,Hudson Street,40.715860,-74.009888,"[{'label': 'display', 'lat': 40.71585960614924...",477,10007,US,New York,NY,United States,"[147 Chambers St (Hudson Street), New York, NY...",4bc238adf8219c744286b410


In [50]:
filtered_df.drop(['Cc', 'City', 'Country', 'Crossstreet', 'Distance', 'Formattedaddress',\
                                        'Labeledlatlngs'], axis=1, inplace=True)

filtered_df.dropna(axis=0, inplace=True)

filtered_df.head()

,Name,Categories,Address,Lat,Lng,Postalcode,State,Id
0,TJ Byrnes Bar and Restaurant,Restaurant,77 Fulton St,40.709233,-74.003747,10038,NY,4b4fdfc8f964a520801827e3
1,Mudville Restaurant & Tap House,Wings Joint,126 Chambers St,40.715336,-74.008881,10007,NY,45e5c256f964a52046431fe3
2,New Shezan Restaurant,Middle Eastern Restaurant,183 Church St,40.715789,-74.007227,10007,NY,4c4890da1879c9b6cce6e143
3,Win Won Restaurant,Chinese Restaurant,10 Liberty Pl,40.709193,-74.009344,10038,NY,4b88199af964a520e6df31e3
4,Amore's Pizza Restaurant,Pizza Place,147 Chambers St,40.715860,-74.009888,10007,NY,4bc238adf8219c744286b410


In [51]:
filtered_df.drop(['Postalcode', 'State', 'Id'], axis=1, inplace=True)

In [57]:
#My dataframe had office category in it so it is being deleted
filtered_df.drop([38], axis=0, inplace=True)

In [58]:
filtered_df

,Name,Categories,Address,Lat,Lng
0,TJ Byrnes Bar and Restaurant,Restaurant,77 Fulton St,40.709233,-74.003747
1,Mudville Restaurant & Tap House,Wings Joint,126 Chambers St,40.715336,-74.008881
2,New Shezan Restaurant,Middle Eastern Restaurant,183 Church St,40.715789,-74.007227
3,Win Won Restaurant,Chinese Restaurant,10 Liberty Pl,40.709193,-74.009344
4,Amore's Pizza Restaurant,Pizza Place,147 Chambers St,40.715860,-74.009888
5,Restaurant at Smyth,American Restaurant,85 W Broadway,40.715206,-74.009132
6,Restaurant New Orleans,Cajun / Creole Restaurant,47 Fulton St,40.707784,-74.004296
7,Roy Rogers Restaurant,Food,250 Broadway,40.712978,-74.007812
8,Kaede Japanese Restaurant,Japanese Restaurant,90 Chambers St,40.714623,-74.007270
9,Chen Hall Restaurant,Food,10 Murray St,40.713341,-74.008095


In [59]:
ratings_rest = [8.6, 8.4, 7.6, 8.0, 9.0, 8.8, 9.0, 7.2, 8.4, 8.0, 9.2, 9.0, 6.8, 7.0, 9.0, 7.4, 8.9, 8.7, 7.8, 8.0, 7.0, 8.0, 8.8, 8.8, 9.4, 8.0, 9.0, 7.8, 8.6, 8.6, 8.2, 8.6, 7.6, 8.0, 8.4, 9.0, 8.0, 8.0, 8.8, 5.0, 7.0, 8.6, 8.8, 8.0, 9.0, 7.2, 9.0, 8.6]
filtered_df['Ratings'] = ratings_rest
filtered_df.head()

,Name,Categories,Address,Lat,Lng,Ratings
0,TJ Byrnes Bar and Restaurant,Restaurant,77 Fulton St,40.709233,-74.003747,8.6
1,Mudville Restaurant & Tap House,Wings Joint,126 Chambers St,40.715336,-74.008881,8.4
2,New Shezan Restaurant,Middle Eastern Restaurant,183 Church St,40.715789,-74.007227,7.6
3,Win Won Restaurant,Chinese Restaurant,10 Liberty Pl,40.709193,-74.009344,8.0
4,Amore's Pizza Restaurant,Pizza Place,147 Chambers St,40.715860,-74.009888,9.0


In [60]:
best_restaurants = filtered_df.loc[filtered_df['Ratings'] >= 9.0]
best_restaurants

,Name,Categories,Address,Lat,Lng,Ratings
4,Amore's Pizza Restaurant,Pizza Place,147 Chambers St,40.715860,-74.009888,9.0
6,Restaurant New Orleans,Cajun / Creole Restaurant,47 Fulton St,40.707784,-74.004296,9.0
10,Restaurant Marc Forgione,New American Restaurant,134 Reade St,40.716380,-74.009629,9.2
11,Aoi Japanese Restaurant,Japanese Restaurant,325 Broadway,40.716120,-74.005278,9.0
14,IL Mattone Restaurant,Italian Restaurant,61 Reade St,40.714730,-74.006716,9.0
25,Gn Restaurant Group,Food,167 Chambers St,40.716357,-74.010477,9.4
27,Pancho Magico Restaurant,Food,213 Pearl St,40.707199,-74.006500,9.0
36,Sumo Japanese Cuisine,Japanese Restaurant,104 John St,40.707714,-74.006226,9.0
46,Takahachi,Sushi Restaurant,145 Duane St,40.716526,-74.008101,9.0
48,Sophie's Cuban Cuisine,Cuban Restaurant,96 Chambers St,40.714803,-74.007656,9.0


In [62]:
restaurant_map = folium.Map(location=[latitude, longitude], zoom_start=14)

#Adding hotels to map
for lat, lng, name, price in zip(hotel_df['Lat'], 
                                 hotel_df['Lng'], 
                                 hotel_df['Name'], 
                                 hotel_df['Price']
                                 ):
    label = '{}, Average Cost is ${}'.format(name, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(restaurant_map)

#Adding restaurants to map
for lat, lng, name in zip(best_restaurants['Lat'], 
                          best_restaurants['Lng'], 
                          best_restaurants['Name']
                          ):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(restaurant_map)
    
restaurant_map

<h4>'Gild Hall, A Thompson Hotel', 'The James Hotel [FiDi]', 'Cosmopolitan Hotel - TriBeCa', 'The Frederick Hotel', 'Smythe Thompson Hotel' are nearby the best restaurants

In [65]:
search_query = 'Subway Station'

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()['response']['venues']

    
restaurants_df = json_normalize(results) # flatten JSON

filtered_columns = ['name', 'categories'] + [col for col in restaurants_df.columns if col.startswith('location')]+ ['id']
filtered_df = restaurants_df.loc[:,filtered_columns]

filtered_df['categories'] = filtered_df.apply(get_category_type, axis=1)

filtered_df.columns = [col.split('.')[-1].capitalize() for col in filtered_df.columns]

filtered_df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,Name,Categories,Address,Crossstreet,Lat,Lng,Labeledlatlngs,Distance,Postalcode,Cc,City,State,Country,Formattedaddress,Neighborhood,Id
0,NYC Green Cart (7th Ave Subway Station),Food Truck,Park Pl,Carlton Ave,40.713926,-74.008882,"[{'label': 'display', 'lat': 40.71392620399956...",276,11217,US,Prospect Heights,NY,United States,"[Park Pl (Carlton Ave), Prospect Heights, NY 1...",NaN,4e0cffc522711665f606edb4
1,MTA Subway - Brooklyn Bridge/City Hall/Chamber...,Metro Station,Centre St,Chambers St,40.712459,-74.005054,"[{'label': 'display', 'lat': 40.71245948565639...",86,10007,US,New York,NY,United States,"[Centre St (Chambers St), New York, NY 10007, ...",NaN,4aaa4e0ef964a520ad5520e3
2,MTA Subway - Fulton St (A/C/J/Z/2/3/4/5),Metro Station,Fulton St,Nassau St,40.710706,-74.009243,"[{'label': 'display', 'lat': 40.71070586655301...",353,10038,US,New York,NY,United States,"[Fulton St (Nassau St), New York, NY 10038, Un...",NaN,4ad78220f964a520960b21e3
3,MTA Subway - City Hall (R/W),Metro Station,Warren St.,at Broadway,40.713394,-74.006934,"[{'label': 'display', 'lat': 40.71339431394821...",107,10038,US,New York,NY,United States,"[Warren St. (at Broadway), New York, NY 10038,...",NaN,4b57b0dff964a520293c28e3
4,MTA Subway - Chambers St (A/C/E),Metro Station,156 Church St,at Chambers St.,40.714086,-74.008584,"[{'label': 'display', 'lat': 40.71408594127328...",264,10007,US,New York,NY,United States,"[156 Church St (at Chambers St.), New York, NY...",NaN,4af180a2f964a52027e121e3


In [66]:
filtered_df.drop(['Cc', 'City', 'Country', 'Crossstreet', 'Distance', 'Formattedaddress',\
                                        'Labeledlatlngs', 'Postalcode', 'Neighborhood', 'Id'], axis=1, inplace=True)
filtered_df.head()

,Name,Categories,Address,Lat,Lng,State
0,NYC Green Cart (7th Ave Subway Station),Food Truck,Park Pl,40.713926,-74.008882,NY
1,MTA Subway - Brooklyn Bridge/City Hall/Chamber...,Metro Station,Centre St,40.712459,-74.005054,NY
2,MTA Subway - Fulton St (A/C/J/Z/2/3/4/5),Metro Station,Fulton St,40.710706,-74.009243,NY
3,MTA Subway - City Hall (R/W),Metro Station,Warren St.,40.713394,-74.006934,NY
4,MTA Subway - Chambers St (A/C/E),Metro Station,156 Church St,40.714086,-74.008584,NY


In [67]:
filtered_df.drop('State', axis=1, inplace=True)
filtered_df.head()

,Name,Categories,Address,Lat,Lng
0,NYC Green Cart (7th Ave Subway Station),Food Truck,Park Pl,40.713926,-74.008882
1,MTA Subway - Brooklyn Bridge/City Hall/Chamber...,Metro Station,Centre St,40.712459,-74.005054
2,MTA Subway - Fulton St (A/C/J/Z/2/3/4/5),Metro Station,Fulton St,40.710706,-74.009243
3,MTA Subway - City Hall (R/W),Metro Station,Warren St.,40.713394,-74.006934
4,MTA Subway - Chambers St (A/C/E),Metro Station,156 Church St,40.714086,-74.008584


In [69]:
subway_map = folium.Map(location=[latitude, longitude], zoom_start=14)

#Adding hotels to map
for lat, lng, name, price in zip(hotel_df['Lat'], 
                                 hotel_df['Lng'], 
                                 hotel_df['Name'], 
                                 hotel_df['Price']
                                 ):
    label = '{}, Average Cost is ${}'.format(name, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(subway_map)

#Adding subway stations to map
for lat, lng, name in zip(filtered_df['Lat'], 
                          filtered_df['Lng'], 
                          filtered_df['Name']
                          ):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        parse_html=False).add_to(subway_map)
    
subway_map

<h3> Surprisingly, every hotel is nearby a subway station. It was not at all expected by me to see such results

<h2>Conclusion

<h3>After going through the notebook, we are able to answer all the questions which were supposed to be answered throughout this project

<h4>Answer 1: 'Four Seasons Hotel New York Downtown', 'The Beekman, A Thompson Hotel', 'Gild Hall, A Thompson Hotel' and 'Cosmopolitan Hotel - TriBeCa'
 

<h4>Answer 2: Map would be shown in the report/presentation

<h4> Answer 3: The best and cheap hotels are</h4><li>The Beekman, A Thompson Hotel</li>
<li>Artezen Hotel</li>
<li>Cosmopolitan Hotel - TriBeCa</li>
<li>Smythe Thompson Hotel</li>
<li>Holiday Inn New York City - Wall Street</li>

<h4> Answer 4: The best hotels nearby airport</h4>
<li>'Four Seasons Hotel New York Downtown' (454 dollars) </li>
<li>'Millennium Hilton' (250 dollars)

<h4>Answer 5: Best hotels nearby restaurants are:</h4>
<li> 'Gild Hall, A Thompson Hotel'</li>
<li>'The James Hotel [FiDi]'</li>
<li>'Cosmopolitan Hotel - TriBeCa'</li>
<li>'The Frederick Hotel'</li>
<li>'Smythe Thompson Hotel'</li>

<h4> Answer 6: All the hotels were found near subway station